In [ ]:
import scipy
import pandas as pd
from pathlib import Path
from utils.classificaton_utils import evaluate_results

# Table and 1.2

In [20]:
extension = '' # '' or '_lemma'

In [21]:

results_1850 = evaluate_results(Path(f'results_1850{extension}'))
results_1900 = evaluate_results(Path(f'results_1900{extension}'))
#results_2000 = evaluate_results(Path('results_2000'))

In [22]:
df_1850 = pd.DataFrame.from_dict(results_1850, orient='index', columns=['precision','recall','fscore'])
df_1900 = pd.DataFrame.from_dict(results_1900, orient='index', columns=['precision','recall','fscore'])
#df_2000 = pd.DataFrame.from_dict(results_2000, orient='index', columns=['precision','recall','fscore'])


In [23]:
df_fscores = pd.concat([df_1850['fscore'],df_1900['fscore']],axis=1) # ,df_2000['fscore']
cols_baselines = ["random", "def_tok_overlap_ranking", "sent_embedding", "w2v_lesk_ranking", "svm_wemb_baseline"]
cols_bert = [c for c in df_fscores.index if not 'ts' in c or 'contrast' in c]
df_fscores = df_fscores.loc[cols_baselines + cols_bert]
print(df_fscores.to_latex())

\begin{tabular}{lrr}
\toprule
{} &  fscore &  fscore \\
\midrule
random                                             &   0.409 &   0.400 \\
def\_tok\_overlap\_ranking                            &   0.577 &   0.586 \\
sent\_embedding                                     &   0.573 &   0.577 \\
w2v\_lesk\_ranking                                   &   0.613 &   0.609 \\
svm\_wemb\_baseline                                  &   0.528 &   0.543 \\
random                                             &   0.409 &   0.400 \\
def\_tok\_overlap\_ranking                            &   0.577 &   0.586 \\
sent\_embedding                                     &   0.573 &   0.577 \\
w2v\_lesk\_ranking                                   &   0.613 &   0.609 \\
svm\_wemb\_baseline                                  &   0.528 &   0.543 \\
bert\_binary\_centroid\_vector\_bert\_base\_-1,-2,-3,... &   0.613 &   0.607 \\
bert\_centroid\_sense\_vector\_bert\_base\_-1,-2,-3,-... &   0.764 &   0.755 \\
bert\_contrast\_vec

In [ ]:
cols_baselines + cols_bert

In [14]:
rows_1850 = [r for r in df_fscores.index if '1850' in r]
rows_1900 = [r for r in df_fscores.index if 'blert' in r]
rows_2000 = [r for r in df_fscores.index if 'bert_base' in r]


In [16]:
diff_1850_2000 = df_1850.loc[df_1850.index.isin(rows_1850)].fscore.values - df_1850.loc[df_1850.index.isin(rows_2000)].fscore.values
diff_1900_2000 = df_1900.loc[df_1900.index.isin(rows_1900)].fscore.values - df_1900.loc[df_1900.index.isin(rows_2000)].fscore.values



# Table 2

In [18]:
df_diff = pd.DataFrame([diff_1850_2000,diff_1900_2000],columns=rows_2000).T
#df_diff['sum'] = df_diff.sum(axis=1)
print(df_diff.to_latex())

\begin{tabular}{lrr}
\toprule
{} &      0 &      1 \\
\midrule
bert\_binary\_centroid\_vector\_bert\_base\_-1,-2,-3,... & -0.024 & -0.008 \\
bert\_centroid\_sense\_vector\_bert\_base\_-1,-2,-3,-... &  0.021 &  0.037 \\
bert\_contrast\_vector\_bert\_base\_-1,-2,-3,-4\_mean    &  0.009 &  0.009 \\
bert\_svm\_vector\_bert\_base\_-1,-2,-3,-4\_mean         &  0.011 &  0.048 \\
bert\_perceptron\_vector\_bert\_base\_-1,-2,-3,-4\_mean  &  0.017 &  0.012 \\
bert\_ml\_perceptron\_vector\_bert\_base\_-1,-2,-3,-4... & -0.005 &  0.020 \\
\bottomrule
\end{tabular}



# Table 3

In [4]:
results_ts = evaluate_results(Path()) # 'results_2000_wo_time_filter'

In [5]:
df_ts = pd.DataFrame.from_dict(results_ts, orient='index', columns=['precision','recall','fscore'])


In [6]:
df_ts['fscore']

bert_centroid_sense_vector_bert_base_-1,-2,-3,-4_mean                 0.753
bert_ts_nearest_centroid_sense_vector_bert_base_-1,-2,-3,-4_mean      0.687
bert_ts_weighted_centroid_sense_vector_bert_base_-1,-2,-3,-4_mean     0.752
bert_centroid_sense_vector_blert_base_-1,-2,-3,-4_mean                0.757
bert_ts_nearest_centroid_sense_vector_blert_base_-1,-2,-3,-4_mean     0.696
bert_ts_weighted_centroid_sense_vector_blert_base_-1,-2,-3,-4_mean    0.756
bert_centroid_sense_vector_bert_1850_-1,-2,-3,-4_mean                 0.752
bert_ts_nearest_centroid_sense_vector_bert_1850_-1,-2,-3,-4_mean      0.693
bert_ts_weighted_centroid_sense_vector_bert_1850_-1,-2,-3,-4_mean     0.753
Name: fscore, dtype: float64

# Curated examples

In [18]:
results_curated_seed = evaluate_results(Path(f'results_curated_1920_seed'))
results_curated_synonym = evaluate_results(Path(f'results_curated_1920_syn'))

In [19]:
df_curated_seed = pd.DataFrame.from_dict(results_curated_seed, orient='index', columns=['precision','recall','fscore'])
df_curated_synonym = pd.DataFrame.from_dict(results_curated_synonym, orient='index', columns=['precision','recall','fscore'])


In [20]:
df_fscore = pd.concat([df_curated_seed['fscore'],df_curated_synonym['fscore']], axis=1)
df_fscore.columns=['vertical','horizontal']
print(df_fscore.to_latex())

\begin{tabular}{lrr}
\toprule
{} &  vertical &  horizontal \\
\midrule
bert\_centroid\_sense\_vector\_bert\_base\_-1,-2,-3,-... &     0.813 &       0.740 \\
bert\_ts\_weighted\_centroid\_sense\_vector\_bert\_bas... &     0.748 &       0.731 \\
bert\_ml\_perceptron\_vector\_bert\_base\_-1,-2,-3,-4... &     0.818 &       0.725 \\
bert\_centroid\_sense\_vector\_blert\_base\_-1,-2,-3,... &     0.819 &       0.750 \\
bert\_ts\_weighted\_centroid\_sense\_vector\_blert\_ba... &     0.769 &       0.756 \\
bert\_ml\_perceptron\_vector\_blert\_base\_-1,-2,-3,-... &     0.757 &       0.733 \\
bert\_centroid\_sense\_vector\_bert\_1850\_-1,-2,-3,-... &     0.794 &       0.755 \\
bert\_ts\_weighted\_centroid\_sense\_vector\_bert\_185... &     0.738 &       0.742 \\
bert\_ml\_perceptron\_vector\_bert\_1850\_-1,-2,-3,-4... &     0.753 &       0.713 \\
\bottomrule
\end{tabular}



# Assess Statistical Significance

In [ ]:
res = evaluate_results(Path("results_1850/"))

selected = "bert_centroid_sense_vector_blert_base_-1,-2,-3,-4_mean"

selected_pred = res[method]["pred"]
print (method, res[method]["metrics"], "\n\nIs the difference significant?\n")

for method,values in res.items():
    if method != selected:
        pred = values["pred"]
        p_value = scipy.stats.ttest_rel(selected_pred,pred)[1]
        if p_value<0.05:
            print (method, values["metrics"], "YES")
        else:
            print (method, values["metrics"], "NO p_value:",round(p_value,5))